# Neural Net Game of Life

A grid of cells, each one has an hp value as a float between 0 and 1, and a color RGB.
each turn, the cell picks a color for itself, and tries to predict the color of all of its neighbors
cell gains hp based on how accurately it predicted neighbors colors
cell loses hp if neighbors predicted its color
a successful color prediction is a "chomp"

it needs memory! use a recursive network. need to use multiclass cross entropy loss function for learning.

cell takes more damage if a high hp cell chomps it?

cells with super high fitness copy their weights over to another cell?

LEARNING RATE tied to HP!!!

hostile environment? all cells take damage over time unless they act correctly to survive?

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [2]:
import random

import torch
import math
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import torch.nn as nn

In [4]:
import numpy as np

In [17]:
color_strings = {
    0: "red",
    1: "green",
    2: "blue",
    3: "grey",
}
color_hexs = {
    0: [0xFF, 0x00, 0x00],
    1: [0x00, 0xFF, 0x00],
    2: [0x00, 0x00, 0xFF],
    3: [0x88, 0x88, 0x88],
}

class Cell():
    def __init__(self):
        self.hp = 0.5
        self.red = np.random.randint(0, 255)
        self.green = np.random.randint(0, 255)
        self.blue = np.random.randint(0, 255)
        self.neighbors = []
        #self.brain = nn.Sequential()

    def change_hp(self, hp_bonus):
        self.hp += hp_bonus
    def set_color(self, color_num):
        self.color = color_num
    def get_color(self):
        return [self.red, self.green, self.blue]
    def get_lr(self):
        return math.cos((math.pi / 2) * self.hp)
    def __repr__(self):
        return f" RGB:{self.red},{self.green},{self.blue} Hp:{self.hp:.2f} "
        # print the state of the cell
    def sense(self, other):
        self.neighbors.append(other.color)
        #fill an array with all the numerical values associated with everything the cell can see



In [28]:
class Board():
    def __init__(self, rows, cols):
        self.cells = []
        self.rows = rows
        self.cols = cols
        for row in range(rows):
            new_row = []
            for col in range(cols):
                new_row.append(Cell())
            self.cells.append(new_row)
        #Assign neighbors
        for row in range(rows):
            for col in range(cols):
                if self.rows > 1:
                    if row == 0:
                        self.cells[row][col].neighbors.append(self.cells[self.rows - 1][col])
                        self.cells[row][col].neighbors.append(self.cells[row + 1][col])
                    if (row > 0) and (row < self.rows - 1):
                        self.cells[row][col].neighbors.append(self.cells[row - 1][col])
                        self.cells[row][col].neighbors.append(self.cells[row + 1][col])
                    if row == self.rows - 1:
                        self.cells[row][col].neighbors.append(self.cells[row - 1][col])
                        self.cells[row][col].neighbors.append(self.cells[0][col])
                if self.cols > 1:
                    if col == 0:
                        self.cells[row][col].neighbors.append(self.cells[row][self.cols - 1])
                        self.cells[row][col].neighbors.append(self.cells[row][col + 1])
                    if (col > 0) and (col < self.cols - 1):
                        self.cells[row][col].neighbors.append(self.cells[row][col - 1])
                        self.cells[row][col].neighbors.append(self.cells[row][col + 1])
                    if col == self.cols - 1:
                        self.cells[row][col].neighbors.append(self.cells[row][col - 1])
                        self.cells[row][col].neighbors.append(self.cells[row][0])
    def __repr__(self):
        board_string = ''
        for row in self.cells:
            for cell in row:
                board_string += str(cell)
                board_string += ', '
            board_string += '\n'
        return board_string
    def show_img(self):
        img_list = []
        for row in range(0, self.rows):
            img_list.append([])
            for column in range(0, self.cols):
                img_list[row].append([])
                img_list[row][column] = (self.cells[row][column].get_color())
        img_array = np.array(img_list)
        plt.imshow(img_array)
        plt.show()
first_board = Board(3, 3)
print(first_board)



 RGB:177,84,88 Hp:0.50 ,  RGB:3,136,237 Hp:0.50 ,  RGB:3,0,119 Hp:0.50 , 
 RGB:84,34,218 Hp:0.50 ,  RGB:84,189,245 Hp:0.50 ,  RGB:130,12,98 Hp:0.50 , 
 RGB:58,131,71 Hp:0.50 ,  RGB:53,165,230 Hp:0.50 ,  RGB:102,58,61 Hp:0.50 , 



In [30]:
first_board.cells[1][1].neighbors

[ RGB:3,136,237 Hp:0.50 ,
  RGB:53,165,230 Hp:0.50 ,
  RGB:84,34,218 Hp:0.50 ,
  RGB:130,12,98 Hp:0.50 ]

In [ ]:
def relu(x):
    return max(0, x)
class MLP():
    def __init__(self):
        hidden_size = 16
        input_size = 3
        output_size = 3
        self.W1 = np.random.uniform(low = -0.01, high = 0.01, size = (hidden_size, input_size))
        self.b1 = np.zeros(hidden_size)
        self.W2 = np.random.uniform(low = -0.01, high = 0.01, size = (output_size, hidden_size))
        self.b2 = np.zeros(output_size)
    def forward(self, X):
        self.Z1 = np.dot(X, self.W1.T) + self.b1
        self.A1 = relu(self.Z1)
        self.Z2 = np.dot(self.A1, self.W2.T) + self.b2
        self.y_hat = np.round(self.Z2)
        return self.y_hat
    def backward(self, X, y):
        self.error = np.mean((y - self.Z2) ** 2)
        self.dZ2 =